# 7.6 Transformer 모델(분류 작업용) 구현

- 클래스 분류의 Transformer 모델을 구현합니다.



※ 이 장의 파일은 Ubuntu 환경에서의 동작을 전제로 하고 있습니다. Windows와 같이 문자 코드가 다른 환경에서는 동작에 주의하십시오.

# 7.6 학습 목표

1.	Transformer 모듈의 구성을 이해한다
2.	LSTM이나 RNN을 사용하지 않아도 CNN 기반의 Transformer로 자연어 처리가 가능한 이유를 이해한다
3.	Transformer를 구현할 수 있다



# 사전 준비
도서의 지시에 따라, 이 장에서 사용하는 데이터를 준비합니다


In [0]:
import math
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchtext

In [0]:
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [0]:
class Embedder(nn.Module):
    '''id로 표시된 단어를 벡터로 변환합니다'''

    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()

        self.embeddings = nn.Embedding.from_pretrained(
            embeddings=text_embedding_vectors, freeze=True)
        # freeze=True에 의해 역전파로 갱신되지 않고, 변하지 않습니다

    def forward(self, x):
        x_vec = self.embeddings(x)

        return x_vec


In [0]:
# 동작 확인

# 이전 절의 DataLoader 등을 취득
from utils.dataloader import get_IMDb_DataLoaders_and_TEXT
train_dl, val_dl, test_dl, TEXT = get_IMDb_DataLoaders_and_TEXT(
    max_length=256, batch_size=24)

# 미니 비치 준비
batch = next(iter(train_dl))

# 모델 구축
net1 = Embedder(TEXT.vocab.vectors)

# 입출력
x = batch.Text[0]
x1 = net1(x)  # 단어를 벡터로

print("입력 텐서 크기: ", x.shape)
print("출력 텐서 크기: ", x1.shape)


입력 텐서 크기:  torch.Size([24, 256])
출력 텐서 크기:  torch.Size([24, 256, 300])


In [0]:
class PositionalEncoder(nn.Module):
    '''입력된 단어의 위치를 나타내는 벡터 정보를 부가'''

    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()

        self.d_model = d_model  # 단어 벡터의 차원수

        # 단어 순서(pos)와 내장 벡터의 차원 위치(i)에 의해 고유하게 정해지는 값의 표를 pe로 작성
        pe = torch.zeros(max_seq_len, d_model)

        # GPU가 사용 가능하면 GPU에 전달하는 것은, 여기서는 생략합니다. 실제 학습시에 사용합니다
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                
                # 오탈자 수정_200510 #79
                # pe[pos, i + 1] = math.cos(pos /
                #                          (10000 ** ((2 * (i + 1))/d_model)))
                pe[pos, i + 1] = math.cos(pos /
                                          (10000 ** ((2 * i)/d_model)))

        # 표 pe의 선두에서, 미니 배치 차원을 더함
        self.pe = pe.unsqueeze(0)

        # 경사를 계산하지 않음
        self.pe.requires_grad = False

    def forward(self, x):

        # 입력 x와 Positonal Encoding을 더함
        # x가 pe보다 작으므로 크게 한다
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret


In [0]:
# 동작 확인

# 모델 구축
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)

# 입출력
x = batch.Text[0]
x1 = net1(x)  # 단어를 벡터로
x2 = net2(x1)

print("입력 텐서 크기: ", x1.shape)
print("출력 텐서 크기: ", x2.shape)


입력 텐서 크기:  torch.Size([24, 256, 300])
출력 텐서 크기:  torch.Size([24, 256, 300])


In [0]:
class Attention(nn.Module):
    '''Transformer는 사실상 멀티 헤드 Attention이지만, 
    쉽게 이해되도록 우선 싱글 Attention로 구현합니다'''

    def __init__(self, d_model=300):
        super().__init__()

        # SAGAN에서는 1dConv를 사용했지만, 이번에는 전결합층에서 특징량을 변환
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)

        # 출력 시에 사용할 전결합층
        self.out = nn.Linear(d_model, d_model)

        # Attention의 크기 조정 변수
        self.d_k = d_model

    def forward(self, q, k, v, mask):
        # 전결합층에서 특징량을 변환
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)

        # Attention 값을 계산한다
        # 각 값을 덧셈하면 너무 커지므로 root(d_k)로 나누어 조절
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)

        # 여기서 mask를 계산
        mask = mask.unsqueeze(1)
        weights = weights.masked_fill(mask == 0, -1e9)

        # softmax로 규격화
        normlized_weights = F.softmax(weights, dim=-1)

        # Attention을 Value와 곱하기
        output = torch.matmul(normlized_weights, v)

        # 전결합층에서 특징량을 변환
        output = self.out(output)

        return output, normlized_weights


In [0]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        '''Attention 층에서 출력을 단순히 전결합층 두 개로 특징량을 변환하는 유닛입니다'''
        super().__init__()

        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x


In [0]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()

        # LayerNormalization층
        # https://pytorch.org/docs/stable/nn.html?highlight=layernorm
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)

        # Attention층
        self.attn = Attention(d_model)

        # Attention 다음의 전결합층 두 개
        self.ff = FeedForward(d_model)

        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        # 정규화와 Attention
        x_normlized = self.norm_1(x)
        output, normlized_weights = self.attn(
            x_normlized, x_normlized, x_normlized, mask)
        
        x2 = x + self.dropout_1(output)

        # 정규화와 전결합층
        x_normlized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normlized2))

        return output, normlized_weights


In [0]:
# 동작 확인

# 모델 구축
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)

# mask 작성
x = batch.Text[0]
input_pad = 1  # 단어 ID에 있어서, '<pad>': 1이므로
input_mask = (x != input_pad)
print(input_mask[0])

# 입출력
x1 = net1(x)  # 단어를 벡터로
x2 = net2(x1)  # Positon 정보를 더한다
x3, normlized_weights = net3(x2, input_mask)  # Self-Attention으로 특징량을 변환

print("입력 텐서 크기: ", x2.shape)
print("출력 텐서 크기: ", x3.shape)
print("Attention 크기: ", normlized_weights.shape)


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
입력 텐서 크기:  torch.Size([24, 256, 300])
출력 텐서 크기:  torch.Size([24, 256, 300])
Attention 크기:  torch.Size([24, 256, 256])


In [0]:
class ClassificationHead(nn.Module):
    '''Transformer_Block의 출력을 사용하여, 마지막에 클래스 분류를 시킨다'''

    def __init__(self, d_model=300, output_dim=2):
        super().__init__()

        # 전결합층
        self.linear = nn.Linear(d_model, output_dim)  # output_dim은 음성, 양성의 두 가지

        # 가중치 초기화
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, x):
        x0 = x[:, 0, :]  # 각 미니 배치의 각 문장의 선두 단어의 특징량(300차원)을 꺼낸다
        out = self.linear(x0)

        return out


In [0]:
# 동작 확인

# 미니 배치 준비
batch = next(iter(train_dl))

# 모델 구축
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=256)
net3 = TransformerBlock(d_model=300)
net4 = ClassificationHead(output_dim=2, d_model=300)

# 입출력
x = batch.Text[0]
x1 = net1(x)  # 단어를 벡터로
x2 = net2(x1)  # Positon 정보를 더한다
x3, normlized_weights = net3(x2, input_mask)  # Self-Attention으로 특징량을 변환
x4 = net4(x3)  # 최종 출력의 0번째 단어를 사용하여, 분류0~1의 스칼라를 출력

print("입력 텐서 사이즈: ", x3.shape)
print("출력 텐서 사이즈: ", x4.shape)


입력 텐서 사이즈:  torch.Size([24, 256, 300])
출력 텐서 사이즈:  torch.Size([24, 2])


In [0]:
# 최종적인 Transformer 모델의 클래스
class TransformerClassification(nn.Module):
    '''Transformer로 클래스 분류'''

    def __init__(self, text_embedding_vectors, d_model=300, max_seq_len=256, output_dim=2):
        super().__init__()

        # 모델 구축
        self.net1 = Embedder(text_embedding_vectors)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)

    def forward(self, x, mask):
        x1 = self.net1(x)  # 단어를 벡터로
        x2 = self.net2(x1)  # Positon 정보를 더한다
        x3_1, normlized_weights_1 = self.net3_1(
            x2, mask)  # Self-Attention으로 특징량을 변환
        x3_2, normlized_weights_2 = self.net3_2(
            x3_1, mask)  # Self-Attention으로 특징량을 변환
        x4 = self.net4(x3_2)  # 최종 출력의 0번째 단어를 사용하여, 분류0~1의 스칼라를 출력
        return x4, normlized_weights_1, normlized_weights_2


In [0]:
# 동작 확인

# 미니 배치 준비
batch = next(iter(train_dl))

# 모델 구축
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)

# 입출력
x = batch.Text[0]
input_mask = (x != input_pad)
out, normlized_weights_1, normlized_weights_2 = net(x, input_mask)

print("출력 텐서 크기: ", out.shape)
print("출력 텐서의 sigmoid: ", F.softmax(out, dim=1))


출력 텐서 크기:  torch.Size([24, 2])
출력 텐서의 sigmoid:  tensor([[0.6980, 0.3020],
        [0.7318, 0.2682],
        [0.7244, 0.2756],
        [0.7135, 0.2865],
        [0.7022, 0.2978],
        [0.6974, 0.3026],
        [0.6831, 0.3169],
        [0.6487, 0.3513],
        [0.7096, 0.2904],
        [0.7221, 0.2779],
        [0.7213, 0.2787],
        [0.7046, 0.2954],
        [0.6738, 0.3262],
        [0.7069, 0.2931],
        [0.7217, 0.2783],
        [0.6837, 0.3163],
        [0.7011, 0.2989],
        [0.6944, 0.3056],
        [0.6860, 0.3140],
        [0.7183, 0.2817],
        [0.7256, 0.2744],
        [0.7288, 0.2712],
        [0.6678, 0.3322],
        [0.7253, 0.2747]], grad_fn=<SoftmaxBackward>)


지금까지의 내용을 "utils" 폴더의 transformer.py에 별도로 저장해 두고, 다음 절에서는 해당 파일을 읽어 사용합니다

끝